In [1]:
import os

In [2]:
%pwd

'/workspaces/End-to-End-MLOps-Bootcamp-Build-Deploy-and-Automate-ML-with-Data-Science-Projects/myFirstNewProject/research'

In [3]:
# changing it the to the root directory
os.chdir("../")

In [4]:
%pwd

'/workspaces/End-to-End-MLOps-Bootcamp-Build-Deploy-and-Automate-ML-with-Data-Science-Projects/myFirstNewProject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
   root_dir: Path
   source_URL: str
   local_data_file: Path
   unzip_dir: Path

In [6]:
from src.my_first_end_to_end_project.constants import *
from src.my_first_end_to_end_project.utils.common_utils import read_yaml, create_directories


class ConfigurationManager:

    # reading the yaml files and creating necessary files
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath =  PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                     ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    # creating the Pipeline
    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

    

In [7]:
## New component class
import urllib.request as request
from src.my_first_end_to_end_project.logger import logger 
import zipfile


class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config= config

    # downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info : {headers}")
        else:
            logger.info(f"The file already exists in the local repository")

    # extract the zip file
    def extract_zip_file(self):
        """
        Objective: To extract the zip file from the local repository
        arguments: NIL
        returns: returns None but loads the unzipped file into local repository
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    


In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2026-01-01 11:32:51 - INFO - common_utils - yaml file: config/config.yaml is loaded successfully 🥳
2026-01-01 11:32:51 - INFO - common_utils - yaml file: params.yaml is loaded successfully 🥳
2026-01-01 11:32:51 - INFO - common_utils - yaml file: schema.yaml is loaded successfully 🥳
2026-01-01 11:32:51 - INFO - common_utils - Created directory Successfuly at: artifacts 🥳
2026-01-01 11:32:51 - INFO - common_utils - Created directory Successfuly at: artifacts/data_ingestion 🥳
2026-01-01 11:32:52 - INFO - 2686935457 - artifacts/data_ingestion/data.zip download! with following info : Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E9AB:3BD445:1